In [1]:
import urllib.request
import json
import networkx as nx
import matplotlib.pyplot as plt

doi = "10.1371/journal.pcbi.1004668"

def get_dois(doi):
    url = "https://api.crossref.org/works/"
    doi_json_data = urllib.request.urlopen(url + doi)
    doi_json_data_content = doi_json_data.read()
    doi_data = json.loads(doi_json_data_content)
    ref_doi_lst = []
    for reference in doi_data["message"]["reference"]:
        try:
            ref_doi_lst.append(reference["DOI"])
        except KeyError:
            pass
    return ref_doi_lst

G = nx.Graph()
G.add_nodes_from(get_dois(doi))
try:
    for new_doi in get_dois(doi):
        G.add_edge(doi, new_doi)
        for more_doi in get_dois(new_doi):
            G.add_edge(new_doi, more_doi)
except:
    pass

In [2]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [3]:
from bokeh.models import Range1d, Plot, HoverTool, MultiLine, Circle
from bokeh.models.graphs import from_networkx, EdgesAndLinkedNodes

In [6]:
plot = Plot(x_range=Range1d(-2, 2), y_range=Range1d(-2, 2))

graph = from_networkx(G, nx.spring_layout, scale=1.8, center=(0,0))
plot.renderers.append(graph)
plot.add_tools(HoverTool(tooltips=None))

graph.node_renderer.glyph.update(size=20, fill_color="orange")

graph.node_renderer.glyph = Circle(size=15, fill_color='#2b83ba')
graph.node_renderer.hover_glyph = Circle(size=15, fill_color='#abdda4')

graph.edge_renderer.glyph = MultiLine(line_alpha=0, line_width=5)
graph.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=5)

graph.inspection_policy = EdgesAndLinkedNodes()
plot.renderers.append(graph)

show(plot)